In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy as sp

from experiment_storage.file_paths import FilePaths
from experiment_storage.storage import EvaluationStorage

parent_dir = "/Users/vdarvariu/experiment_data/ates"
# parent_dir = '/Users/shunee/github/ATES2022/experiment_data'

instances = ["C101200", "R101200", "O101200"]
which="olaoodgnntestincrscale"
# which="olaoodgnntestincrbudget"

train_size = 20
test_sizes = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
exclude_seeds = []
only_best_seed = False

fp_out = FilePaths(parent_dir, 'aggregate', setup_directories=False)


In [ ]:
all_rows = []

for instance in instances:
    for test_size in test_sizes:
        exp_id = f"{instance}_{which}{test_size}"
        fp_in = FilePaths(parent_dir, exp_id, setup_directories=False)
        storage = EvaluationStorage(fp_in)
        results = storage.get_evaluation_data(exp_id)
        results = [r for r in results if (r["is_best_hyps"] and r["agent_seed"] not in exclude_seeds)]
        for r in results:
            r['num_custs'] = test_size
            r['instance'] = instance

        all_rows.extend(results)


In [ ]:
gnn_df = pd.DataFrame(all_rows)
if only_best_seed:
    pivot = pd.pivot_table(gnn_df, values='cummulative_reward', columns=['algorithm'], index=['agent_seed'])
    best_seeds = pivot.idxmax(axis=0)
    algo_names = list(pivot.columns)
    for algo_name in algo_names:
        best_seed = best_seeds[algo_name]
        gnn_df = gnn_df.drop(gnn_df[(gnn_df['algorithm'] == algo_name) & (gnn_df['agent_seed'] != best_seed)].index)


In [ ]:
gnn_df

In [ ]:

all_rows = []
analysis_scales = [2,4,6,8,10]
which_s = 'olascaleredone'

for instance in instances:
    for scale in analysis_scales:
        exp_id = f"{instance}_{which_s}{scale}"
        fp_in = FilePaths(parent_dir, exp_id, setup_directories=False)
        storage = EvaluationStorage(fp_in)
        results = storage.get_evaluation_data(exp_id)
        results = [r for r in results if (r["is_best_hyps"] and r["agent_seed"] not in exclude_seeds)]
        for r in results:
            r['scale'] = scale
            r['instance'] = instance

        all_rows.extend(results)

scale_df = pd.DataFrame(all_rows)


In [ ]:
scale_df

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

font_scale=3.25
sns.set(font_scale=font_scale)
plt.rc('font', family='serif')
mpl.rcParams['text.usetex'] = True
mpl.rcParams["lines.linewidth"] = 6

dims = ((8.26 * 1.05) * len(instances), (2) * 8.26)

fig, axes = plt.subplots(2, len(instances), squeeze=False, figsize=dims, sharex="row", sharey="row")

inst_display={"C101200": "Solomon C instance", "R101200": "Solomon R instance", "O101200": "Solomon RC instance"}
agent_display={"dqn": "DQN", "vrwmdp": "LRW", "randommdp": "RAN"}


for i, plot_x in enumerate(["num_custs", "scale"]):
    for j, inst_name in enumerate(instances):
        ax = axes[i][j]

        data_df = gnn_df if plot_x == "num_custs" else scale_df
        data_df = data_df[data_df['instance'] == inst_name]

        data_df.replace(agent_display, inplace=True)

        sns.lineplot(data=data_df, x=plot_x, y="cummulative_reward", hue='algorithm', ax=ax)

        handles, labels = ax.get_legend_handles_labels()

        if i == 0:
            ax.set_xticks(list(range(10, 101, 10)))
            ax.set_xlabel('Number of customers')
        else:
            ax.set_xticks([2,4,6,8,10])
            ax.set_xlabel('Destroy scale')

        if j == 0:
            ax.set_ylabel("Cumulative reward", fontsize="medium")
        else:
            ax.set_ylabel("")

        if i==0:
            ax.set_title(inst_display[inst_name])

        if ax.legend_ is not None:
            ax.legend_.remove()

        if i == 1 and j == 2:
            ax.legend(handles[0:], labels[0:], loc='lower right', fontsize="medium")

filename = ("gnns_scale" if which == "olaoodgnntestincrscale" else "gnnb_scale") + (".pdf" if not only_best_seed else "_bestseed.pdf")
fig.tight_layout()
plt.savefig(fp_out.figures_dir / filename, bbox_inches="tight")

In [ ]:
from copy import deepcopy

all_rows = []
which_t = 'olascaleredone4'

temp_dict ={"t0d01": 0.01,
            "t0d1": 0.1,
            "t1": 1,
            "t5": 5,
            "t10": 10,
            "t100": 100
            }

for instance in instances:
    exp_id = f"{instance}_{which_t}"
    fp_in = FilePaths(parent_dir, exp_id, setup_directories=False)
    storage = EvaluationStorage(fp_in)
    results = storage.get_evaluation_data(exp_id, task_type="alns")
    proc_results = []
    for r in results:
        algo_name = r["algorithm"]
        if not algo_name.startswith("dqn"):
            continue

        else:
            temp_val = temp_dict[algo_name.split("_")[1]]
            rmin = {"metric": "obj_min", "val": r["obj_min"], "temperature": temp_val}
            ravg = {"metric": "obj_avg", "val": r["obj_avg"], "temperature": temp_val}


        proc_results.append(rmin)
        proc_results.append(ravg)

    # for r in results:
    #     r['instance'] = instance

    all_rows.extend(proc_results)

temp_df = pd.DataFrame(all_rows)



In [ ]:
temp_df

In [ ]:
dims = ((8.26 * 1.05), (0.9) * 8.26)
metric_display={"obj_min": "min($F$)", "obj_avg": "avg($F$)"}
temp_df.replace(metric_display, inplace=True)

fig, axes = plt.subplots(1, 1, squeeze=False, figsize=dims, sharex="row", sharey="row")
ax = axes[0][0]
ax.set_xscale('symlog')
sns.lineplot(data=temp_df, x="temperature", y="val", ax=ax, palette="flare", hue="metric")
handles, labels = ax.get_legend_handles_labels()

ax.set_xlabel("DQN temperature $\\tau$")
ax.set_ylabel("")
ax.set_ylim([100, 315])

if ax.legend_ is not None:
    ax.legend_.remove()

ax.legend(handles[0:], labels[0:], loc='lower right', fontsize="small")
filename="dqn_temp.pdf"
fig.tight_layout()
plt.savefig(fp_out.figures_dir / filename, bbox_inches="tight")